# Import the Data

In [1]:
import torchvision
import torch
import numpy as np
import copy 



normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

transform_data = torchvision.transforms.Compose([
            torchvision.transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
            normalize,
        ])

dataset = torchvision.datasets.ImageFolder('images',transform=transform_data)    
dataset

Dataset ImageFolder
    Number of datapoints: 516
    Root Location: images
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None

# Break Data into Training and Test Sets with Batches

In [3]:
validation_size = .10
num_train = len(dataset)
indices = list(range(num_train))
split = int(np.floor(validation_size * num_train))
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_idx)

batch_size = 128


train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler,num_workers=1)      
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler,num_workers=1)

print("Data Loaders Initialized")

Data Loaders Initialized


# Load a Pretrained Model

In [4]:
import torchvision.models as models

pretrained_model = models.resnet101(pretrained=True) #load in the renet pretrained model
pretrained_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

# Freeze the Pretrained Model's Weights

In [7]:
for param in list(pretrained_model.children()):
    param.require_grad = False

print("All model weights are frozen")

All model weights are frozen


# Change the Classification Layer

In [9]:
number_classes = 2

num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = torch.nn.Linear(num_ftrs, number_classes)
print("Last Layer Overwritten")

Last Layer Overwritten


# Train the New Model

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)
pretrained_model = pretrained_model.to(device)

loss_function = torch.nn.CrossEntropyLoss()

# Observe that only the fc parameters are being optimized
optimizer = torch.optim.SGD(pretrained_model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 3 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

cpu


# Define Functions to Train and Test the model

In [13]:
def evaluate(data_loader,epoch,train=True):
    losses = []
    if train:
        pretrained_model.train()
        for batch_idx, (data, target) in enumerate(data_loader):
            if not torch.cuda.is_available(): 
                data, target = torch.autograd.Variable(data).cuda(), torch.autograd.Variable(target).cuda()
            else: 
                data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
            
            optimizer.zero_grad()
            output = pretrained_model(data)
            loss = loss_function(output, target)
            
            print("Train Epoch: ",epoch, "Batch :", batch_idx, "Loss: ", np.float(loss.data))
            
            #check the order of these
            loss.backward()
            optimizer.step()
            
    else:
        pretrained_model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(data_loader):
                
                if  not torch.cuda.is_available(): 
                    data, target = torch.autograd.Variable(data).cuda(), torch.autograd.Variable(target).cuda()
                else: 
                    data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)

                output = pretrained_model(data)
                loss = loss_function(output, target)
                losses.append(np.float(loss.data))
    
    return np.mean(losses)


# Train the Model

In [14]:
torch.cuda.empty_cache()
torch.set_num_threads(12)

best_loss = float("inf")
for epoch in range(5):
    train_loss = evaluate(train_loader,epoch,train=True)
    test_loss = evaluate(test_loader,epoch,train=False)
    if test_loss < best_loss:
        best_model = copy.deepcopy(pretrained_model)
        best_loss = test_loss
    scheduler.step()
    print("Epoch: ", epoch+1, "Test Loss: ", test_loss)

RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

# View Final Evaluation

In [ ]:
correct_predictions = 0
predictions_made = 0

pretrained_model.eval()

for batch_idx, (data, target) in enumerate(test_loader):
    with torch.no_grad():
        if not torch.cuda.is_available(): 
            data, target = torch.autograd.Variable(data).cuda(), torch.autograd.Variable(target).cuda()

        else: 
            data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)

        output = best_model(data)

        predicted = output.data.max(1)[1]
        correct_predictions += predicted.eq(target.data).cpu().sum()
        predictions_made += len(target.data)

print("Test Accuracy :", float(correct_predictions) / predictions_made)

# Save the Model

In [ ]:
torch.save(best_model, "pretrained_model.pt")
print("model saved")

# Load the Model

In [ ]:
pretrained_model = torch.load("pretrained_model.pt")
pretrained_model

# Make a Prediction

In [ ]:
from PIL import Image
torch.cuda.empty_cache()

pretrained_model.eval()

img = Image.open("hilary.jpg")

normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

preprocess = torchvision.transforms.Compose([
   torchvision.transforms.Resize ((224,224)),
   torchvision.transforms.ToTensor(),
   normalize
])

img_tensor = preprocess(img)
img_tensor.unsqueeze_(0)

if not torch.cuda.is_available(): 
    img_variable = torch.autograd.Variable(img_tensor).cuda()
else:
    img_variable = torch.autograd.Variable(img_tensor)
    
output = pretrained_model(img_variable)
predicted = output.data.max(1)[1]

if predicted == 0:
    print("female")
else:
    print("male")